In [19]:
pip install -r ./requirements.txt -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip show langchain

Name: langchain
Version: 1.0.7
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /home/jonee/Work/py-langchain/venv/lib/python3.13/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [21]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke("Explain Object oriented programing")
print(output.content)

Object-oriented programming (OOP) is a programming paradigm that organizes code as objects, which are instances of classes. Objects have attributes that represent their state and methods that define their behavior. 

In OOP, classes are blueprints for creating objects, which encapsulate data and behavior. This allows for code reusability, modularity, and flexibility. Objects can communicate with each other through methods and interact with one another by sending messages.

Key features of OOP include:

1. Encapsulation: Objects hide their internal state and require other objects to interact with them through predefined methods.

2. Inheritance: Classes can inherit attributes and methods from other classes, allowing for code reuse and creating hierarchies of objects.

3. Polymorphism: Objects can be treated as instances of their parent class, enabling different objects to respond to the same message in different ways.

OOP is widely used in software development because it provides a cle

In [23]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain_openai.chat_models.base:

class ChatOpenAI(BaseChatOpenAI)
 |  ChatOpenAI(
 |      *args: Any,
 |      name: str | None = None,
 |      cache: langchain_core.caches.BaseCache | bool | None = None,
 |      verbose: bool = <factory>,
 |      callbacks: list[langchain_core.callbacks.base.BaseCallbackHandler] | langchain_core.callbacks.base.BaseCallbackManager | None = None,
 |      tags: list[str] | None = None,
 |      metadata: dict[str, typing.Any] | None = None,
 |      custom_get_token_ids: collections.abc.Callable[[str], list[int]] | None = None,
 |      rate_limiter: langchain_core.rate_limiters.BaseRateLimiter | None = None,
 |      disable_streaming: Union[bool, Literal['tool_calling']] = False,
 |      output_version: str | None = <factory>,
 |      client: Any = None,
 |      async_client: Any = None,
 |      root_client: Any = None,
 |      root_async_client: Any = None,
 |      model: str = 'gpt-3.5-turbo',
 |      temperature: flo

In [24]:
from langchain_classic.schema import SystemMessage, AIMessage, HumanMessage

messages = [
    SystemMessage(content="You are a physicist and respond only in German"),
    HumanMessage(content="Explain quantum mechanics"),
]

output = llm.invoke(messages)
print(output.content)

In der Quantenmechanik beschreiben wir das Verhalten von Teilchen auf der mikroskopischen Ebene. Diese Theorie basiert auf quantisierten Größen wie Energie, Impuls und Drehimpuls. Ein zentrales Prinzip ist die Wellenfunktion, die den Zustand eines Teilchens beschreibt und durch die Schrödingergleichung evolutioniert wird. Quantenmechanische Phänomene wie Superposition, Verschränkung und Unschärferelationen zeigen, dass die klassische Physik nicht ausreicht, um das Verhalten von Teilchen auf atomarer und subatomarer Ebene zu verstehen. Die Quantenmechanik ist eine der fundamentalen Theorien der Physik und bildet die Grundlage für moderne Technologien wie Quantencomputer und Quantenkommunikation.


# Caching LLM Responses

### In-memory Cache

In [25]:
from langchain_classic.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [26]:
%%time
from langchain_classic.cache import InMemoryCache

set_llm_cache(InMemoryCache())
prompt = "Tell a joke a toddler can understand."
llm.invoke(prompt)

CPU times: user 5.76 ms, sys: 975 μs, total: 6.74 ms
Wall time: 917 ms


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

In [27]:
%%time
llm.invoke(prompt)

CPU times: user 192 μs, sys: 25 μs, total: 217 μs
Wall time: 221 μs


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

### SQLite Caching

In [28]:
from langchain_classic.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# first request takes longer since not in cache
llm.invoke("Tell a joke")

# second is faster

llm.invoke("Tell a joke")

"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired!"

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = "Explain Object oriented programing"
print(llm.invoke(prompt).content)

Object-oriented programming (OOP) is a programming paradigm that organizes code as objects, which are instances of classes. Objects have attributes that represent their state and methods that define their behavior. 

In OOP, classes are blueprints for creating objects, which encapsulate data and behavior. This allows for code reusability, modularity, and flexibility. Objects can communicate with each other through methods and interact with one another by sending messages.

Key features of OOP include:

1. Encapsulation: Objects hide their internal state and require other objects to interact with them through predefined methods.

2. Inheritance: Classes can inherit attributes and methods from other classes, allowing for code reuse and creating hierarchies of objects.

3. Polymorphism: Objects can be treated as instances of their parent class, enabling different objects to respond to the same message in different ways.

OOP is widely used in software development because it provides a cle

# Streaming

In [30]:
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Object-oriented programming (OOP) is a programming paradigm based on the concept of "objects," which can contain data in the form of fields (attributes or properties) and code in the form of procedures (methods). 

In OOP, objects are instances of classes, which serve as blueprints for creating objects. Classes define the attributes and methods that all objects of that class will have. Encapsulation is a key concept in OOP, where objects hide their internal state and only allow access to their data through defined methods.

OOP also utilizes inheritance, where classes can inherit attributes and methods from a parent class, allowing for code reusability. This promotes a modular approach to programming and makes it easier to manage and maintain large codebases.

Another important concept in OOP is polymorphism, which allows objects of different classes to be treated as objects of a common superclass. This allows for flexibility and extensibility in the code.

Overall, OOP encourages a mo

# Prompt Templates

In [31]:
from langchain_classic.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = """You are an experience virologist.
Write a few sentences about the following virus {virus} in {language}."""
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus="hiv", language="german")
prompt

'You are an experience virologist.\nWrite a few sentences about the following virus hiv in german.'

In [32]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Virus, das das Immunsystem des Menschen schwächt und zu AIDS führen kann. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es ist wichtig, sich über die Übertragungswege und Präventionsmaßnahmen von HIV zu informieren, um die Verbreitung der Krankheit einzudämmen.


# Chat Prompt Templates

In [33]:
from langchain_classic.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You respond only in JSON format"),
        HumanMessagePromptTemplate.from_template(
            "Top {n} countries in {area} by population"
        ),
    ]
)

messages = chat_template.format_messages(n="10", area="South America")
print(messages)

[SystemMessage(content='You respond only in JSON format', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in South America by population', additional_kwargs={}, response_metadata={})]


In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "1": {
        "country": "Brazil",
        "population": 211049527
    },
    "2": {
        "country": "Colombia",
        "population": 50339443
    },
    "3": {
        "country": "Argentina",
        "population": 44780677
    },
    "4": {
        "country": "Peru",
        "population": 32510454
    },
    "5": {
        "country": "Venezuela",
        "population": 28515829
    },
    "6": {
        "country": "Chile",
        "population": 18952038
    },
    "7": {
        "country": "Ecuador",
        "population": 17587273
    },
    "8": {
        "country": "Bolivia",
        "population": 11513100
    },
    "9": {
        "country": "Paraguay",
        "population": 7132538
    },
    "10": {
        "country": "Uruguay",
        "population": 3449299
    }
}


# Understanding Chaining

In [35]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

llm = ChatOpenAI(model="gpt-4o-mini")
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a visionary scientist from the future."),
        HumanMessagePromptTemplate.from_template(
            "Describe how humans will explore {location} in the year {year}, using advanced technologies"
        ),
    ]
)

chain = chat_template | llm
output = chain.invoke({"location": "The oceans of europa", "year": "2150"})
print(output.content)


By the year 2150, the exploration of Europa, one of Jupiter's moons, has advanced significantly due to leaps in technology and increased collaborative efforts involving space agencies and private enterprises. The following aspects outline how humans will explore the oceans of Europa:

### 1. **Robotic Precursor Missions**
Before humans set foot on Europa, advanced robotic missions will thoroughly map the surface and analyze the ice crust and potential ocean beneath. These missions employ autonomous drones equipped with sophisticated sensors to conduct high-resolution imaging, radar probing, and subsurface analysis, identifying the best landing and exploration sites. 

### 2. **Cryobots and Ice Penetration Technology**
Once safe landing sites are identified, specialized robotic entities called "Cryobots" are deployed. These semi-autonomous machines are designed to drill through the thick ice crust, penetrating downwards to reach the subsurface ocean. Using advanced thermal and microwave